<a href="https://colab.research.google.com/github/prezOPERAH/NLP-Helper/blob/master/Chaitbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import yaml
import numpy as np 
import tensorflow as tf
from tensorflow.keras import layers, activations, models, preprocessing, utils 
import pandas as pd

##Loading Data

In [2]:
with open('ai.yml') as f:
  docs=yaml.load(f)
  
  
print(docs)


questions=list()
answers=list()


conversations= docs['conversations']
for con in conversations:  #we have 2 cases as in case of a comma appearing in the dict leading to 3 arguments the 2nd and 3rd are joined in the answers
    if len(con)>2:
      questions.apppend(con[0])
      replies=con[1:]
      ans= ''
       
      
      for rep in replies:
        ans+=' ' + rep 
      answers.append(ans)
       
      
    elif len(con)>1:
      questions.append(con[0])  #incase of only 2 arguments we simply assign conv[0]->ques and conv[1]->ans
      answers.append(con[1])


print(questions)
print(answers)

{'categories': ['AI', 'artificial intelligence'], 'conversations': [['What is AI?', 'Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.'], ['What is AI?', 'AI is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind.'], ['Are you sentient?', 'Sort of.'], ['Are you sentient?', "By the strictest dictionary definition of the word 'sentience', I may be."], ['Are you sentient?', "Even though I'm a construct I do have a subjective experience of the universe, as simplistic as it may be."], ['Are you sapient?', "In all probability, I am not.  I'm not that sophisticated."], ['Are you sapient?', 'Do you think I am?'], ['Are you sapient?', 'How would you feel about me if I told you I was?'], ['Are you sapient?', 'No.'], ['What language are you written in?', 'Python.'], ['What language are you written in?', 'I am written in Python.'], ['You sound like Data', "Yes I am

##Preprocessing the text

In [12]:
tokenizer= preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions)

tokenized_questions=tokenizer.texts_to_sequences(questions)
length_list=list()
for token_seq in tokenized_questions:
  length_list.append(len(token_seq))
max_input_length=np.array(length_list).max()

padded_questions=preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=max_input_length, padding='post')
encoder_input_data= np.array(padded_questions)
#ques_word_dict= tokenizer.word_index
#num_ques_tokens=len(ques_word_dict)

#encoder input data?
new_answers=list()


for line in answers:
  new_answers.append ('<START> ' +line + '<END>' )



tokenizer.fit_on_texts(new_answers)
tokenized_answers=tokenizer.texts_to_sequences(new_answers)


length_list = list()
for token_seq in tokenized_answers:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()

padded_answers= preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=max_output_length,padding='post')
decoder_input_data=np.array(padded_answers)

word_dict=tokenizer.word_index
num_tokens=len(word_dict)+1
final_answers=list()

for lines in tokenized_answers :
  final_answers.append(lines[1:])



padded_answers=preprocessing.sequence.pad_sequences(final_answers, maxlen=max_output_length, padding='post')
one_hot_lines=utils.to_categorical(padded_answers, num_tokens)
final_answers=np.array(one_hot_lines)









##Building the Model

In [13]:
import tensorflow as tf
encoder_inputs= tf.keras.layers.Input(shape=(None , ))
encoder_embedding= tf.keras.layers.Embedding(num_tokens, 200, mask_zero=True)(encoder_inputs)
encoder_outputs , state_h, state_c= tf.keras.layers.LSTM( 200 , return_state=True)(encoder_embedding)
encoder_states= [ state_h, state_c]


decoder_inputs= tf.keras.layers.Input(shape=(None , ))
decoder_embedding=tf.keras.layers.Embedding(num_tokens,200, mask_zero=True)(decoder_inputs)

decoder_lstm=tf.keras.layers.LSTM(200, return_state=True, return_sequences=True)

decoder_outputs , _ , _ =decoder_lstm(decoder_embedding, initial_state=encoder_states)

decoder_dense= tf.keras.layers.Dense(num_tokens, activation=tf.keras.activations.softmax)
output= decoder_dense (decoder_outputs)


model=tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 200)    80200       input_6[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 200)    80200       input_7[0][0]                    
_______________________________________________________________________________________

In [18]:
model.fit([encoder_input_data, decoder_input_data], final_answers,batch_size=250, epochs=100)
model.save('model.h5')

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5288
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5189
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5058
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4931
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4856
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4799
Epoch 7/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4806
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4726
Epoch 9/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4711
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4580
Epoch 11/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4636
Epoch 12/100
1/1 [==============================] - 0s 1ms/step - loss: 0.4586
Epoch 13/100
1/1 [==============================] - 0s 1ms/st

##Uing GloVe to fine tune Classification

In [19]:
def make_inference_models():
  encoder_model=tf.keras.models.Model(encoder_inputs, encoder_states)

  decoder_state_input_h=tf.keras.layers.Input(shape=(200, ))
  decoder_state_input_c=tf.keras.layers.Input(shape=(200, ))

  decoder_states_inputs= [decoder_state_input_h, decoder_state_input_c]

  decoder_outputs, state_h, state_c =decoder_lstm(
      decoder_embedding, initial_state=deocoder_states_inputs)
  decoder_states=[state_h, state_c]
  decoder_outputs= decoder_dense(decoder_outputs)
  decoder_model=tf.keras.models.Model(
      [deocoder_inputs] + decoder_states_inputs,
      [decoder_outputs] + decoder_states)
  
  return encoder_model , decoder_model
  

##Using Our Model

In [20]:
def str_to_tokens( sentence : str):
  words=sentence.lower().split()
  tokens_list=list()
  for word in words:
    toekens_list.append(ques_word_dict[word])
    return preprocessing.sequence.pad_sequences([tokens_list], maxlen=max_input_length, padding='post' )

In [21]:

enc_model , dec_model = make_inference_models()

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter sentence : ' ) ) )
    #states_values = enc_model.predict( encoder_input_data[ epoch ] )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


NameError: ignored